#### Module Importation

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

%reload_ext autoreload
%autoreload 
from HelperFunctions import missingValuesInfo

from sklearn.impute import SimpleImputer
from fancyimpute import KNN
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

#### Helper Function

In [5]:
def imputer_score(imputer,estimator,x_miss,y):
    x_imputed=imputer.fit_transform(x_miss)
    impute_scores=cross_val_score(estimator,x_imputed,y,scoring='neg_mean_squared_error',cv=5)
    return -1*impute_scores.mean()

# Import Data

In [13]:
df=pd.read_csv('lean_df_5.csv',index_col='id')

/home/auscheng/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# Variable Type Definition

In [7]:
df_number = df.select_dtypes(include = 'number')
df_object = df.select_dtypes(include = 'object')
df_category = df.select_dtypes(include = 'category')
df_boolean = df.select_dtypes(include = 'bool')
df_datetime = df.select_dtypes(include = 'datetime')
df_timedelta = df.select_dtypes(include = 'timedelta')
#######################################################
nominal_var=list(df_object.columns)
ordinal_var=list(df_number.columns)
continuous_var=list(df_number.columns)
time_var=list(df_datetime.columns)

# Instantiate Imputer 

In [8]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_median = SimpleImputer(missing_values=np.nan, strategy= 'median')
imp_mode = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
imp_const = SimpleImputer(missing_values=np.nan,fill_value='none',strategy="constant")
imp_const_n1 = SimpleImputer(missing_values=np.nan,fill_value=-999,strategy="constant")
imp_KNN = KNN(k=3)
# imp_MICE = IterativeImputer(random_state=0)


In [24]:
missingdf=missingValuesInfo(df)
missingdf

,Total,Percent
mths_since_last_derog_record,1899411,84.11
mths_since_last_major_derog,1678024,74.31
mths_since_last_delinq,1157161,51.24
install_util,1067769,47.28
active_install_frac,909200,40.26
all_util,865694,38.33
inq_last_12m,865477,38.33
emp_length,146661,6.49
active_card_frac,75835,3.36
pct_acc_nvr_dlq,70317,3.11


In [29]:
missinglist=list(missingdf.index)
impute_with_zero_list=['pub_rec_bankruptcies','tax_liens',\
             'delinq_amnt','total_acc','derog_records',\
             'delinq_2yrs','collections_12_mths_ex_med',\
             'chargeoff_within_12_mths','pub_rec_bankruptcies']
impute_with_neg999=list(set(missinglist)-set(impute_with_zero_list))

In [32]:
for item in impute_with_zero_list:
    df.loc[df.loc[:,item].isnull(),item]=0

In [33]:
for item in impute_with_neg999:
    df.loc[df.loc[:,item].isnull(),item]=-999

In [34]:
missingValuesInfo(df)

,Total,Percent


In [35]:
df.to_csv('final_df.csv')

In [36]:
df.head(20)

,funded_amnt,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,mths_since_last_delinq,mths_since_last_derog_record,derog_records,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,collection_amt,install_util,all_util,rev_credit,inq_last_12m,chargeoff_within_12_mths,delinq_amnt,mths_since_RecentAcc_opened,all_accs_120days+_PastDue_ever,accs_90days+_PastDue_24m,accs_opened_past_12m,pct_acc_nvr_dlq,pub_rec_bankruptcies,tax_liens,total_credit,install_credit,fico,Outstanding_mortgage_debt,revol_frac,install_frac,mort_frac,card_frac,active_card_frac,active_revol_frac,active_install_frac,open_revol_frac,good_acc_frac,loan_duration,profit,RANDOM
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
68407277,3600.0,36,13.99,C,C4,10+ years,MORTGAGE,55000.0,Not Verified,2015-12-01,Fully Paid,debt_consolidation,190xx,PA,5.91,0.0,2003-08-01,30.0,-999.0,0.0,29.7,13.0,w,0.0,30.0,Individual,0.0,722.0,36.0,34.0,9300.0,4.0,0.0,0.0,3.0,2.0,0.0,3.0,76.9,0.0,0.0,178050.0,13734.0,677.0,137158.0,0.692308,0.230769,0.076923,0.555556,0.400000,0.444444,0.666667,0.444444,0.538462,37.566667,4421.723917,0.952901
68355089,24700.0,36,11.99,C,C1,10+ years,MORTGAGE,65000.0,Not Verified,2015-12-01,Fully Paid,small_business,577xx,SD,16.06,1.0,1999-12-01,6.0,-999.0,0.0,19.2,38.0,w,0.0,-999.0,Individual,0.0,0.0,73.0,29.0,111800.0,6.0,0.0,0.0,2.0,0.0,0.0,2.0,97.4,0.0,0.0,314017.0,24667.0,717.0,164921.0,0.710526,0.157895,0.105263,0.629630,0.294118,0.185185,0.166667,0.740741,0.578947,6.100000,25679.660000,0.743162
68341763,20000.0,60,10.78,B,B4,10+ years,MORTGAGE,63000.0,Not Verified,2015-12-01,Fully Paid,home_improvement,605xx,IL,10.78,0.0,2000-08-01,-999.0,-999.0,0.0,56.2,18.0,w,0.0,-999.0,Joint App,0.0,0.0,73.0,65.0,14000.0,1.0,0.0,0.0,14.0,0.0,0.0,0.0,100.0,0.0,0.0,218418.0,14877.0,697.0,171003.0,0.388889,0.333333,0.277778,0.571429,0.500000,0.428571,0.166667,0.571429,0.333333,18.266667,22705.924294,0.119479
66310712,35000.0,60,14.85,C,C5,10+ years,MORTGAGE,110000.0,Source Verified,2015-12-01,Current,debt_consolidation,076xx,NJ,17.06,0.0,2008-09-01,-999.0,-999.0,0.0,11.6,17.0,w,0.0,-999.0,Individual,0.0,0.0,70.0,45.0,67300.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,100.0,0.0,0.0,381215.0,18000.0,787.0,249274.0,0.764706,0.117647,0.058824,0.769231,0.400000,0.384615,0.500000,0.769231,0.764706,38.600000,31464.010000,0.050002
68476807,10400.0,60,22.45,F,F1,3 years,MORTGAGE,104433.0,Source Verified,2015-12-01,Fully Paid,major_purchase,174xx,PA,25.37,1.0,1998-06-01,12.0,-999.0,0.0,64.5,35.0,w,0.0,-999.0,Individual,0.0,0.0,84.0,78.0,34000.0,3.0,0.0,0.0,4.0,0.0,0.0,4.0,96.6,0.0,0.0,439570.0,88097.0,697.0,235962.0,0.542857,0.285714,0.171429,0.473684,0.444444,0.315789,0.300000,0.368421,0.342857,7.100000,11740.500000,0.731115
68426831,11950.0,36,13.44,C,C3,4 years,RENT,34000.0,Source Verified,2015-12-01,Fully Paid,debt_consolidation,300xx,GA,10.20,0.0,1987-10-01,-999.0,-999.0,0.0,68.4,6.0,w,0.0,-999.0,Individual,0.0,0.0,99.0,76.0,12900.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,100.0,0.0,0.0,16900.0,4000.0,692.0,0.0,0.666667,0.333333,0.000000,0.500000,1.000000,0.750000,0.500000,1.000000,0.833333,17.233333,13708.948530,0.072298
68476668,20000.0,36,9.17,B,B2,10+ years,MORTGAGE,180000.0,Not Verified,2015-12-01,Fully Paid,debt_consolidation,550xx,MN,14.67,0.0,1990-06-01,49.0,-999.0,0.0,84.5,27.0,f,0.0,-999.0,Individual,0.0,0.0,63.0,74.0,94200.0,1.0,0.0,0.0,10.0,0.0,0.0,2.0,96.3,0.0,0.0,388852.0,46452.0,682.0,243596.0,0.592593,0.259259,0.148148,0.312500,0.800000,0.375000,0.285714,0.562500,0.444444,11.200000,21393.800000,0.244540
67275481,20000.0,36,8.49,B,B1,10+ years,MORTGAGE,85000.0,Not Verified,2015-12-01,Fully Paid,major_purchase,293xx,SC,17.61,1.0,1999-02-01,3.0,-999.0,0.0,5.7,15.0,w,0.0,3.0,Individual,0.0,0.0,75.0,55.0,14500.0,2.0,0.0,0.0,13.0,1.0,1.0,0.0,93.3,0.0,0.0,193390.0,36144.0,707.0,113664.0,0.

In [9]:
# df[continuous_var]=imp_mean.fit_transform(df[continuous_var])
# df[ordinal_var]=imp_mean.fit_transform(df[ordinal_var])
# df[nominal_var]=imp_const_n1.fit_transform(df[nominal_var])
# df[continuous_var]=imp_KNN.fit_transform(df[continuous_var])
# df[continuous_var]=imp_MICE.fit_transform(df[continuous_var])

/home/auscheng/anaconda3/lib/python3.7/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


MemoryError: 

In [ ]:
x=df_number.drop('SalePrice',axis=1)
y=df_number.SalePrice
regressor=RandomForestRegressor(n_estimators=100,random_state=0)
classifier=RandomForestClassifier(n_estimators=100,random_state=0)
imputer_score(imp_MICE,regressor,x,y)

In [ ]:
# from sklearn.metrics import accuracy_score, log_loss
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC, LinearSVC, NuSVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="rbf", C=0.025, probability=True),
#     NuSVC(probability=True),
#     DecisionTreeClassifier(),
#     RandomForestClassifier(),
#     AdaBoostClassifier(),
#     GradientBoostingClassifier()
#     ]
# for classifier in classifiers:
#     pipe = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', classifier)])
#     pipe.fit(X_train, y_train)   
#     print(classifier)
#     print("model score: %.3f" % pipe.score(X_test, y_test))

In [38]:
# from sklearn.impute import MissingIndicator
# x=np.array([[np.nan, 1, 3],[4, 0, np.nan],[8, 1, 0]])
# indicator = MissingIndicator()
# indicator.fit_transform(x)  

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').